# SEQUENCE ALIGNMENT

In [1]:

import matplotlib.pyplot as plt
from numpy.random import choice


def cost(letter1, letter2):
    """ Calculate the cost or score of the alignment of 2 letters.
    Parameters:
    ----------
    letter1: character
    letter2: character
    Return:
    ------
    score: int
        score of the alignment (-1 if the 2 letters are different, 2 if they are similar)
    """
    if letter1 == letter2:
        return 2
    else:
        return -1


def addgap(seq, gaplist, verbose=0):
    """ Insert gaps into sequence.
    Parameters:
    ----------
    seq: string of characters
        sequence of nucleotides
    gaplist: list of int
        list of gaps to insert successively in the sequence seq
    Return:
    ------
    newseq: string of characters
        the modified sequence with inserted gaps
    """
    newseq = seq[:]
    for g in gaplist:
        if g > len(newseq):
            print("gap postion bigger than sequence length -> gap inserted at the end of the sequence")
        newseq = newseq[:g] + '_' + newseq[g:]
        if verbose > 0:
            print("gap introduced in {}th position -> new sequence equals {}.".format(g, newseq))
    return newseq


def substitution(seq, pos, verbose=0, value=None, alphabet=["A", "C", "G", "T"]):
    """ Induce a mutation of the sequence by substituting a letter.
    Parameters:
    ----------
    seq: string of characters
        sequence of nucleotides
    pos: int in [-len(seq), len(seq)]
        position of the mutation
    verbose: int (default=0)
        level of verbosity
    value: None or alphabet item (default=None)
        new letter induced by the mutation.
        if None, an item different from the initial one is randomly chosen in alphabet.
    alphabet: list of characters (default=["A", "C", "G", "T"])
        list of nucleotides to consider in sequences
    Return:
    ------
    seqr: string
        the modified sequence containing the mutation
    """
    seqr = list(seq[:])
    alphabis = alphabet.copy()
    alphabis.remove(seqr[pos])
    if value == None:
        seqr[pos] = choice(alphabis, 1)[0]
    elif value in alphabet:
        seqr[pos] = value
    else:
        return "error"
    seqr = "".join(seqr)
    if verbose > 0:
        print(seqr)
    return seqr


def insertion(seq, pos, verbose=0, value=None, alphabet=["A", "T", "G", "C"]):
    """ Induce a mutation of the sequence by inserting a new letter.
    Parameters:
    ----------
    seq: string of characters
        sequence of nucleotides
    pos: int in [-len(seq), len(seq)]
        position of the mutation
    verbose: int (default=0)
        level of verbosity
    value: None or alphabet item (default=None)
        new letter induced by the mutation.
        if None, an item different from the initial one is randomly chosen in alphabet.
    alphabet: list of characters (default=["A", "C", "G", "T"])
        list of nucleotides to consider in sequences
    Return:
    ------
    seqr: string
        the modified sequence containing the mutation
    """
    seqr = seq[:]
    if value is None:
        value = choice(alphabet, 1)[0]
    seqr = seqr[:pos] + value + seqr[pos:]
    seqr = "".join(seqr)
    if verbose > 0:
        print(seqr)
    return seqr


def deletion(seq, pos, verbose=0):
    """ Induce a mutation of the sequence by deleting a letter.
    Parameters:
    ----------
    seq: string of characters
        sequence of nucleotides
    pos: int in [-len(seq), len(seq)]
        position of the mutation
    verbose: int (default=0)
        level of verbosity
    Return:
    ------
    seqr: string
        the modified sequence containing the mutation
    """
    seqr = list(seq[:])
    del seqr[pos]
    seqr = "".join(seqr)
    if verbose > 0:
        print(seqr)
    return seqr


def print_table(table):
    """ Display the score or path matrix as a table.
    Parameters:
    ----------
    table: matrix of floats or int
        score or path matrix
    Return:
    ------
    none
    """
    plt.figure()
    tb = plt.table(cellText=table.astype(int), loc=(0,0), cellLoc='center')
    tc = tb.properties()['child_artists']
    for cell in tc:
        cell.set_height(1/table.shape[0])
        cell.set_width(1/table.shape[1])
    ax = plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()


if __name__ == "__main__":

    alphabet = ["A", "C", "G", "T"]

    print("TEST ADDGAP FUNCTION")
    seq = choice(alphabet, 7)
    seq = ''.join(seq)
    gaplist = [1, 4, 7, 0, 0]
    newseq = addgap(seq, gaplist)
    print("The initial sequence equals: {} and the gaplist equals: {}. "
          "The final sequence equals: {}.".format(seq, gaplist, newseq))

    print("TEST COST FUNCTION")
    for i in range(5):
        l1 = choice(alphabet)
        l2 = choice(alphabet)
        print("The cost of ({}, {}) equals {}.".format(l1, l2, cost(l1, l2)))

TEST ADDGAP FUNCTION
The initial sequence equals: CCACAGG and the gaplist equals: [1, 4, 7, 0, 0]. The final sequence equals: __C_CA_CA_GG.
TEST COST FUNCTION
The cost of (T, C) equals -1.
The cost of (A, G) equals -1.
The cost of (G, A) equals -1.
The cost of (T, C) equals -1.
The cost of (A, T) equals -1.


# LOCAL ALIGNMENT

In [2]:
import numpy as np
import pandas as pd

class local_alignment:
    def __init__(self, match_award, mismatch_penalty, gap_penalty):
        self.match_award = match_award
        self.mismatch_penalty = mismatch_penalty
        self.gap_penalty = gap_penalty
        self.order_stack = []
        self.index_stack = []
        
    def check_match(self, data, i, j):
        if data.index[i] == data.columns[j]: return self.match_award
        else: return self.mismatch_penalty
    
    def make_matrix(self, first_seq, second_seq):
        first_seq2 = ["-"]
        second_seq2 = ["-"]
        first_seq2.extend(list(first_seq))
        second_seq2.extend(list(second_seq))
        
        data = [np.zeros(len(first_seq2), dtype='int32') for i in second_seq2]
        data = pd.DataFrame(data, columns=first_seq2, index=second_seq2)
        
        # save the route to trace back
        flag = []
        
        # fill out the matrix
        for i in range(1, data.shape[0]):
            temp = []
            for j in range(1, data.shape[1]):
                value_score = [data.iloc[i,j-1]+self.gap_penalty,
                               data.iloc[i-1,j-1]+self.check_match(data, i, j),
                               data.iloc[i-1,j]+self.gap_penalty,
                               0]
                data.iloc[i,j] = max(value_score)
                temp.append([data.iloc[i,j] == item for item in value_score])
            flag.append(temp)
        return data, flag
    
    def extract_max(self, data):
        temp = np.array(data.max())
        col = np.where(temp == temp.max())[0][0]
        temp = data.iloc[:,col]
        row = np.where(temp == temp.max())[0][0]
        return data.max().max(), row, col
        
    def extract_position(self, i, j, position):
        if position == 0: return [i,j-1]
        elif position == 1: return [i-1,j-1]
        elif position == 2: return [i-1,j]
        else: return None
    
    def stack_route(self, i, j, data, flag, point):
        term = flag[i][j]
        cnt = flag[i][j].count(True)
        
        # stack order
        for k in range(cnt): self.order_stack.append(point)
        
        # stack index
        if cnt == 1:
            pos = term.index(True)
            self.index_stack.append(self.extract_position(i, j, pos))
        elif cnt == 2:
            term = np.array(term)
            term = np.where(term == True)[0]
            self.index_stack.append(self.extract_position(i, j, term[0]))
            self.index_stack.append(self.extract_position(i, j, term[1]))
        elif cnt == 3:
            for k in range(3): self.index_stack.append(self.extract_position(i, j, k))
        else:
            print("error: 'cnt' is out of range (1,3)")
        return None
    
    def backtracking(self, data, flag, row, col):
        i = row - 1
        j = col - 1
        point = 0
        key = 0
        seq_dic = {str(key):[[i,j]]}
        
        # make index list
        while i >= 0 or j >= 0:
            self.stack_route(i, j, data, flag, point)
            self.order_stack.pop()
            term = self.index_stack.pop()
            i, j = term[0], term[1]
            if data.iloc[i+1,j+1] == 0: break
            seq_dic[str(key)].append(term)
            point += 1
            if len(self.order_stack) != 0 and i < 0 or j < 0:
                point = self.order_stack.pop()
                term = self.index_stack.pop()
                i, j = term[0], term[1]
                if data.iloc[i+1,j+1] == 0: break
                key += 1
                seq_dic[str(key)] = seq_dic[str(key-1)][:point+1]
                seq_dic[str(key)].append(term)
        
        # convert index list to name list
        key = 0
        for value in seq_dic.values():
            row, col = [i for i in zip(*value)]
            row = [row[i] if row[i] != row[i+1] else -1 for i in range(len(row)-1)] + [row[-1]]
            col = [col[i] if col[i] != col[i+1] else -1 for i in range(len(col)-1)] + [col[-1]]
            
            row = [data.index[i+1] for i in row]
            col = [data.columns[i+1] for i in col]
            row = "".join(row)[::-1]
            col = "".join(col)[::-1]
            seq_dic[str(key)] = [col, row]
            key += 1
        return seq_dic
    
if __name__ == "__main__":
    # input data
    print("----------- LOCAL ALIGNMENT -----------")
    #first_seq = input("Please enter the first nucleotide sequence: ")
    #second_seq = input("Please enter the second nucleotide sequence: ")

    first_seq = "ATTTATCCGCCGATTCC"
    second_seq = "TCCGCCGATGCACG"

    # assign award and penalties
    match_award = 3
    mismatch_penalty = -1
    gap_penalty = -2
    
    # local alignment
    process = local_alignment(match_award, mismatch_penalty, gap_penalty)
    data, flag = process.make_matrix(first_seq, second_seq)
    score, row, col = process.extract_max(data)
    seq_dic = process.backtracking(data, flag, row, col)
    for key in seq_dic.keys():
        print("< case",int(key)+1,">")
        print("score:", score)
        print(seq_dic[key][0])
        print(seq_dic[key][1])ATTTATCCGCCGATTCC

----------- LOCAL ALIGNMENT -----------
< case 1 >
score: 30
TCCGCCGATTC-C
TCCGCCGATGCAC


In [3]:
data

,-,A,T,T,T,A,T,C,C,G,C,C,G,A,T,T,C,C
-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
T,0,0,3,3,3,1,3,1,0,0,0,0,0,0,3,3,1,0
C,0,0,1,2,2,2,1,6,4,2,3,3,1,0,1,2,6,4
C,0,0,0,0,1,1,1,4,9,7,5,6,4,2,0,0,5,9
G,0,0,0,0,0,0,0,2,7,12,10,8,9,7,5,3,3,7
C,0,0,0,0,0,0,0,3,5,10,15,13,11,9,7,5,6,6
C,0,0,0,0,0,0,0,3,6,8,13,18,16,14,12,10,8,9
G,0,0,0,0,0,0,0,1,4,9,11,16,21,19,17,15,13,11
A,0,3,1,0,0,3,1,0,2,7,9,14,19,24,22,20,18,16
T,0,1,6,4,3,1,6,4,2,5,7,12,17,22,27,25,23,21


In [4]:
'''A Python implementation of the Smith-Waterman algorithm for local alignment
of nucleotide sequences.
'''

import argparse
import os
import re
import sys
import unittest


# These scores are taken from Wikipedia.
# en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm
match    = 2
mismatch = -1
gap      = -2
seq1     = "ATGCAGGCGGTGCGCAACGCGGGCAGCCGCTTTCTGCGCAGCTGGACCTGGCCGCAGACCGCGGGCGTGGTGGCGCGCACCCCGGCGGGCACCATTTGCA"
seq2     = "ATGCATGGCCTGATTATGGGCGGCCTGGAAGTGATTAGCATTACCGATAACACCCCGATTCCGCATAACGGCTGCCGCCCGCGCAAAGCGCGCAAACTGA"


def main():
    try:
        parse_cmd_line()
    except ValueError as err:
        print('error:', err)
        return

    # The scoring matrix contains an extra row and column for the gap (-), hence
    # the +1 here.
    rows = len(seq1) + 1
    cols = len(seq2) + 1

    # Initialize the scoring matrix.
    score_matrix, start_pos = create_score_matrix(rows, cols)

    # Traceback. Find the optimal path through the scoring matrix. This path
    # corresponds to the optimal local sequence alignment.
    seq1_aligned, seq2_aligned = traceback(score_matrix, start_pos)
    assert len(seq1_aligned) == len(seq2_aligned), 'aligned strings are not the same size'

    # Pretty print the results. The printing follows the format of BLAST results
    # as closely as possible.
    alignment_str, idents, gaps, mismatches = alignment_string(seq1_aligned, seq2_aligned)
    alength = len(seq1_aligned)
    print()
    print(' Identities = {0}/{1} ({2:.1%}), Gaps = {3}/{4} ({5:.1%})'.format(idents,
          alength, idents / alength, gaps, alength, gaps / alength))
    print()
    for i in range(0, alength, 60):
        seq1_slice = seq1_aligned[i:i+60]
        print('Query  {0:<4}  {1}  {2:<4}'.format(i + 1, seq1_slice, i + len(seq1_slice)))
        print('             {0}'.format(alignment_str[i:i+60]))
        seq2_slice = seq2_aligned[i:i+60]
        print('Sbjct  {0:<4}  {1}  {2:<4}'.format(i + 1, seq2_slice, i + len(seq2_slice)))
        print()


def parse_cmd_line():
    '''Parse the command line arguments.

    Create a help menu, take input from the command line, and validate the
    input by ensuring it does not contain invalid characters (i.e. characters
    that aren't the bases A, C, G, or T).
    '''



    seq1 = "ATAGACGACATACAGACAGCATACAGACAGCATACAGA"
    seq2 = "TTTAGCATGCGCATATCAGCAATACAGACAGATACG"


def create_score_matrix(rows, cols):
    '''Create a matrix of scores representing trial alignments of the two sequences.

    Sequence alignment can be treated as a graph search problem. This function
    creates a graph (2D matrix) of scores, which are based on trial alignments
    of different base pairs. The path with the highest cummulative score is the
    best alignment.
    '''
    score_matrix = [[0 for col in range(cols)] for row in range(rows)]

    # Fill the scoring matrix.
    max_score = 0
    max_pos   = None    # The row and columbn of the highest score in matrix.
    for i in range(1, rows):
        for j in range(1, cols):
            score = calc_score(score_matrix, i, j)
            if score > max_score:
                max_score = score
                max_pos   = (i, j)

            score_matrix[i][j] = score

    assert max_pos is not None, 'the x, y position with the highest score was not found'

    return score_matrix, max_pos


def calc_score(matrix, x, y):
    '''Calculate score for a given x, y position in the scoring matrix.

    The score is based on the up, left, and upper-left neighbors.
    '''
    similarity = match if seq1[x - 1] == seq2[y - 1] else mismatch

    diag_score = matrix[x - 1][y - 1] + similarity
    up_score   = matrix[x - 1][y] + gap
    left_score = matrix[x][y - 1] + gap

    return max(0, diag_score, up_score, left_score)


def traceback(score_matrix, start_pos):
    '''Find the optimal path through the matrix.

    This function traces a path from the bottom-right to the top-left corner of
    the scoring matrix. Each move corresponds to a match, mismatch, or gap in one
    or both of the sequences being aligned. Moves are determined by the score of
    three adjacent squares: the upper square, the left square, and the diagonal
    upper-left square.

    WHAT EACH MOVE REPRESENTS
        diagonal: match/mismatch
        up:       gap in sequence 1
        left:     gap in sequence 2
    '''

    END, DIAG, UP, LEFT = range(4)
    aligned_seq1 = []
    aligned_seq2 = []
    x, y         = start_pos
    move         = next_move(score_matrix, x, y)
    while move != END:
        if move == DIAG:
            aligned_seq1.append(seq1[x - 1])
            aligned_seq2.append(seq2[y - 1])
            x -= 1
            y -= 1
        elif move == UP:
            aligned_seq1.append(seq1[x - 1])
            aligned_seq2.append('-')
            x -= 1
        else:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[y - 1])
            y -= 1

        move = next_move(score_matrix, x, y)

    aligned_seq1.append(seq1[x - 1])
    aligned_seq2.append(seq1[y - 1])

    return ''.join(reversed(aligned_seq1)), ''.join(reversed(aligned_seq2))


def next_move(score_matrix, x, y):
    diag = score_matrix[x - 1][y - 1]
    up   = score_matrix[x - 1][y]
    left = score_matrix[x][y - 1]
    if diag >= up and diag >= left:     # Tie goes to the DIAG move.
        return 1 if diag != 0 else 0    # 1 signals a DIAG move. 0 signals the end.
    elif up > diag and up >= left:      # Tie goes to UP move.
        return 2 if up != 0 else 0      # UP move or end.
    elif left > diag and left > up:
        return 3 if left != 0 else 0    # LEFT move or end.
    else:
        # Execution should not reach here.
        raise ValueError('invalid move during traceback')


def alignment_string(aligned_seq1, aligned_seq2):
    # Build the string as a list of characters to avoid costly string
    # concatenation.
    idents, gaps, mismatches = 0, 0, 0
    alignment_string = []
    for base1, base2 in zip(aligned_seq1, aligned_seq2):
        if base1 == base2:
            alignment_string.append('|')
            idents += 1
        elif '-' in (base1, base2):
            alignment_string.append(' ')
            gaps += 1
        else:
            alignment_string.append(':')
            mismatches += 1

    return ''.join(alignment_string), idents, gaps, mismatches


def print_matrix(matrix):
    '''Print the scoring matrix.

    ex:
    0   0   0   0   0   0
    0   2   1   2   1   2
    0   1   1   1   1   1
    0   0   3   2   3   2
    0   2   2   5   4   5
    0   1   4   4   7   6
    '''
    for row in matrix:
        for col in row:
            print('{0:>4}'.format(col))
        print()


class ScoreMatrixTest(unittest.TestCase):
    '''Compare the matrix produced by create_score_matrix() with a known matrix.'''
    def test_matrix(self):
        #                -   A   C   A   C   A   C   T   A
        known_matrix = [[0,  0,  0,  0,  0,  0,  0,  0,  0],  # -
                        [0,  2,  1,  2,  1,  2,  1,  0,  2],  # A
                        [0,  1,  1,  1,  1,  1,  1,  0,  1],  # G
                        [0,  0,  3,  2,  3,  2,  3,  2,  1],  # C
                        [0,  2,  2,  5,  4,  5,  4,  3,  4],  # A
                        [0,  1,  4,  4,  7,  6,  7,  6,  5],  # C
                        [0,  2,  3,  6,  6,  9,  8,  7,  8],  # A
                        [0,  1,  4,  5,  8,  8, 11, 10,  9],  # C
                        [0,  2,  3,  6,  7, 10, 10, 10, 12]]  # A

        global seq1, seq2
        seq1 = 'AGCACACA'
        seq2 = 'ACACACTA'
        rows = len(seq1) + 1
        cols = len(seq2) + 1

        matrix_to_test, max_pos = create_score_matrix(rows, cols)
        self.assertEqual(known_matrix, matrix_to_test)


if __name__ == '__main__':
    sys.exit(main())


 Identities = 32/114 (28.1%), Gaps = 43/114 (37.7%)

Query  1     ATGCAG-G-CGGTGCGCAACG-C-GGGC-AGCCG-CTTTCTGCGCAGCT-GGACCT--GG  60  
             |||||:   |:: |:::|::  : ||:  :|::: :     |:::|||: :: ||:  ::
Sbjct  1     ATGCATG-GCCT-GATTATG-GGCGGC-CTGGAAGT-----GATTAGCATTA-CCGATAA  60  

Query  61    CC-GC-AGAC--C-GCGGGC-GTGGTGGC-GCGC-AC--CCCG--GCG--GGCA  114 
             |: :  :||:    ||:::: |: :||:  ||:  :|    ::   ||    ||
Sbjct  61    CACC-CCGATTC-CGCATAACGG-CTGC-CGCC-CGCGC--AAAG-CGCG--CA  114 



SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# GLOBAL ALIGNMENT

In [5]:
import numpy as np
import pandas as pd

class global_alignment:
    def __init__(self, match_award, mismatch_penalty, gap_penalty):
        self.match_award = match_award
        self.mismatch_penalty = mismatch_penalty
        self.gap_penalty = gap_penalty
        self.order_stack = []
        self.index_stack = []
        
    def check_match(self, data, i, j):
        if data.index[i] == data.columns[j]: return self.match_award
        else: return self.mismatch_penalty
    
    def make_matrix(self, first_seq, second_seq):
        first_seq2 = ["-"]
        second_seq2 = ["-"]
        first_seq2.extend(list(first_seq))
        second_seq2.extend(list(second_seq))
        
        data = [np.zeros(len(first_seq2), dtype='int32') for i in second_seq2]
        data = pd.DataFrame(data, columns=first_seq2, index=second_seq2)
        
        # fill 1st row and 1st column with gap_penalty
        data.iloc[0,:] = [self.gap_penalty*i for i in range(data.shape[1])]
        data.iloc[:,0] = [self.gap_penalty*i for i in range(data.shape[0])]
        
        # save the route to trace back
        flag = []
        
        # fill out the matrix
        for i in range(1, data.shape[0]):
            temp = []
            for j in range(1, data.shape[1]):
                value_score = [data.iloc[i,j-1]+self.gap_penalty,
                               data.iloc[i-1,j-1]+self.check_match(data, i, j),
                               data.iloc[i-1,j]+self.gap_penalty]
                data.iloc[i,j] = max(value_score)
                temp.append([data.iloc[i,j] == item for item in value_score])
            flag.append(temp)
        return data, flag
    
    def extract_position(self, i, j, position):
        if position == 0: return [i,j-1]
        elif position == 1: return [i-1,j-1]
        else: return [i-1,j]
    
    def stack_route(self, i, j, data, flag, point):
        term = flag[i][j]
        cnt = flag[i][j].count(True)
        
        # stack order
        for k in range(cnt): self.order_stack.append(point)
        
        # stack index
        if cnt == 1:
            pos = term.index(True)
            self.index_stack.append(self.extract_position(i, j, pos))
        elif cnt == 2:
            term = np.array(term)
            term = np.where(term == True)[0]
            self.index_stack.append(self.extract_position(i, j, term[0]))
            self.index_stack.append(self.extract_position(i, j, term[1]))
        elif cnt == 3:
            for k in range(3): self.index_stack.append(self.extract_position(i, j, k))
        else:
            print("error: 'cnt' is out of range (1,3)")
        return None
    
    def backtracking(self, data, flag):
        i = len(flag) - 1
        j = len(flag[0]) - 1
        maximum = max(i, j)
        point = 0
        key = 0
        seq_dic = {str(key):[[i,j]]}
        
        # make index list
        while point < maximum or i >= 0 or j >= 0:
            self.stack_route(i, j, data, flag, point)
            self.order_stack.pop()
            term = self.index_stack.pop()
            i, j = term[0], term[1]
            if i < 0 or j < 0: break
            seq_dic[str(key)].append(term)
            point += 1
            if point > maximum and len(self.order_stack) != 0:
                point = self.order_stack.pop()
                term = self.index_stack.pop()
                i, j = term[0], term[1]
                key += 1
                seq_dic[str(key)] = seq_dic[str(key-1)][:point+1]
                seq_dic[str(key)].append(term)
        
        # convert index list to name list
        key = 0
        for value in seq_dic.values():
            row, col = [i for i in zip(*value)]
            row = [row[i] if row[i] != row[i+1] else -1 for i in range(len(row)-1)] + [row[-1]]
            col = [col[i] if col[i] != col[i+1] else -1 for i in range(len(col)-1)] + [col[-1]]
            
            row = [data.index[i+1] for i in row]
            col = [data.columns[i+1] for i in col]
            row = "".join(row)[::-1]
            col = "".join(col)[::-1]
            seq_dic[str(key)] = [col, row]
            key += 1
        return seq_dic

if __name__ == "__main__":
    # input data
    print("----------- global alignment -----------")
    #first_seq = input("Please enter the first nucleotide sequence: ")
    #second_seq = input("Please enter the second nucleotide sequence: ")
    
    first_seq = "ATTTATCCGCCGATTCC"
    second_seq = "ATGCAYGGNYTNATHAT"
    
    # assign award and penalties
    match_award = 3
    mismatch_penalty = -1
    gap_penalty = -2
    
    # global alignment
    process = global_alignment(match_award, mismatch_penalty, gap_penalty)
    data, flag = process.make_matrix(first_seq, second_seq)
    score = data.iloc[-1,-1]
    seq_dic = process.backtracking(data, flag)
    for key in seq_dic.keys():
        print("< case",int(key)+1,">")
        print("score:", score)
        print(seq_dic[key][0])
        print(seq_dic[key][1])

----------- global alignment -----------
< case 1 >
score: 4
ATTTATCCGCCG-ATTCC
ATGCA-YGGNYTNATHAT
< case 2 >
score: 4
ATTTATCCGCCG-ATTCC
ATGCAY-GGNYTNATHAT


In [6]:
data

,-,A,T,T,T,A,T,C,C,G,C,C,G,A,T,T,C,C
-,0,-2,-4,-6,-8,-10,-12,-14,-16,-18,-20,-22,-24,-26,-28,-30,-32,-34
A,-2,3,1,-1,-3,-5,-7,-9,-11,-13,-15,-17,-19,-21,-23,-25,-27,-29
T,-4,1,6,4,2,0,-2,-4,-6,-8,-10,-12,-14,-16,-18,-20,-22,-24
G,-6,-1,4,5,3,1,-1,-3,-5,-3,-5,-7,-9,-11,-13,-15,-17,-19
C,-8,-3,2,3,4,2,0,2,0,-2,0,-2,-4,-6,-8,-10,-12,-14
A,-10,-5,0,1,2,7,5,3,1,-1,-2,-1,-3,-1,-3,-5,-7,-9
Y,-12,-7,-2,-1,0,5,6,4,2,0,-2,-3,-2,-3,-2,-4,-6,-8
G,-14,-9,-4,-3,-2,3,4,5,3,5,3,1,0,-2,-4,-3,-5,-7
G,-16,-11,-6,-5,-4,1,2,3,4,6,4,2,4,2,0,-2,-4,-6
N,-18,-13,-8,-7,-6,-1,0,1,2,4,5,3,2,3,1,-1,-3,-5


In [7]:
import numpy as np
import itertools
import utils

from utils import cost, addgap


def align_nw_2by2(seq1, seq2):
    """ Perform 2 by 2 sequence alignment with the Needleman and Wunsch algorithm.
    Parameters:
    ----------
    seq1: string of characters of len=n1
        sequence of nucleotides
    seq2: string of characters of len=n2
        sequence of nucleotides
    Return:
    ------
    scores: matrix of floats of dim (n1+1, n2+1)
        
    paths: matrix of floats of dim (n1+1, n2+1)
        
    aligned: list of 2 strings
        contains the aligned seq1 and seq2 respectively 
    L: list of int
        list of the positions of the inserted gaps
    """
    n1, n2 = len(seq1), len(seq2)
    # initialization: path matrix, score matrix, aligned sequences list, gaps list
    paths = np.zeros((n1 + 1, n2 + 1))
    scores = np.zeros((n1 + 1, n2 + 1))
    aligned = ["", ""]
    L = []
    for i in range(1, n1 + 1):  # browsing seq1 indexes
        scores[i, 0] = scores[i - 1, 0] - 3
        paths[i, 0] = 3
        for j in range(1, n2 + 1):  # browsing seq2 indexes
            scores[0, j] = scores[0, j - 1] - 3
            paths[0, j] = 1
            c1 = scores[i - 1, j - 1] + cost(seq1[i - 1], seq2[j - 1])
            c2 = scores[i - 1, j] - 3
            c3 = scores[i, j - 1] - 3
            scores[i, j] = max(c1, c2, c3)
            if scores[i, j] == c1:
                paths[i, j] = 2
            elif scores[i, j] == c2:
                paths[i, j] = 3
            elif scores[i, j] == c3:
                paths[i, j] = 1

    while i != 0 or j != 0:
        if paths[i, j] == 1:
            aligned[0] += '_'
            aligned[1] += seq2[j - 1]
            j = j - 1
        elif paths[i, j] == 2:
            aligned[0] += seq1[i - 1]
            aligned[1] += seq2[j - 1]
            j = j - 1
            i = i - 1
        elif paths[i, j] == 3:
            aligned[0] += seq1[i - 1]
            aligned[1] += '_'
            L.append(j)  # save gaps introduced by alignment
            i = i - 1
    aligned[0] = aligned[0][::-1]
    aligned[1] = aligned[1][::-1]
    return scores, paths, aligned, L


def align_star_multiple(seqs, verbose=2):
    """ Perform 2 by 2 sequence alignment with the Needleman and Wunsch algorithm.
    Parameters:
    ----------
    seqs: list of strings of characters
        list of sequences of nucleotides to align
    verbose: int (default=1)
        level of verbosity
    Return:
    ------
    res: 
    """
    # enumerate all possible combinations of 2 sequences
    pairs = list(itertools.combinations(list(np.arange(len(seqs))), 2))
    if verbose > 1:
        print("pairs =", pairs)
    # compute 2 by 2 scores for all combinaisons
    scores2a2 = pairs.copy()
    for ip, p in enumerate(pairs):
        scores2a2[ip] = align_nw_2by2(seqs[p[0]], seqs[p[1]])[0][-1, -1]
    if verbose > 1:
        print("scores2a2 =", scores2a2)
    # compute global scores for each sequence as sum of 2by2 scores
    scores = [0]*len(seqs)
    for i in range(len(seqs)):
        scores[i] = np.sum([sc for sci, sc in enumerate(scores2a2) if i in pairs[sci]])
    if verbose > 1:
        print("global scores =", scores)
    # choose pivot sequence as the one with the best (highest) global score
    imax = scores.index(max(scores))  # index of the pivot sequence
    pivot = seqs[imax]  # pivot sequence
    if verbose > 0:
        print("pivot sequence = ", pivot)
    # align all w.r.t pivot
    if verbose > 0:
        print("initial seqs =")
        print("\n".join(str(s) for s in seqs))
    res = seqs.copy()
    # align all sequences all together
    for ir in range(len(res)):
        # align sequences to pivot
        if res[ir] != pivot:  # the pivot sequence does not need to be aligned to itself
            alignment = align_nw_2by2(res[ir], res[imax])
            res[ir], res[imax] = alignment[2]  # result of the alignment
            L = alignment[3]  # result list gap position to consider
            if res[imax] != pivot:  # if new gaps were introduced into the pivot sequence with the 2by2 alignment
                pivot = res[imax]  # update the pivot
            # and update all the other sequences by inserting the same gaps (positions stored in L)
            for i in range(ir):
                if res[i] != pivot:
                    res[i] = addgap(res[i], L)
    if verbose > 0:
        print("res aligned to pivot =\n")
        print("\n".join(str(r) for r in res))
    return res


if __name__ == "__main__":
    from numpy.random import choice

    ex = "ATGAGAT"
    print(addgap(ex, [5, 1, 0, 0, 0]))

    print("EXAMPLE 0 : 2a2\n")
    print(align_nw_2by2("ATGAGAT", "AGGAGAGT"))
    print("EXAMPLE 1 : COURSE PAGE 16\n")
    align_star_multiple(["ATGAGAT", "AGGAGAGT", "GGAGG", "AGGGAGT", "AGAAC"])
    print("EXAMPLE 2 : TEST ON SYNTHESIZED DATA\n")
    alphabet = ["A", "C", "G", "T"]
    seq0 = choice(alphabet, 7)
    seq0 = ''.join(seq0)
    print(seq0)
    # seq 1 = substitution middle
    seq1 = seq0[:]
    alphabet.remove(seq1[3])
    seq1 = list(seq1)
    seq1[3] = choice(alphabet, 1)[0]
    seq1 = "".join(seq1)
    print(seq1)
    # seq 2 = deletion pos 5 and end
    seq2 = list(seq0[:])
    seq2.remove(seq2[4])
    seq2.remove(seq2[5])
    seq2 = "".join(seq2)
    print(seq2)
    # seq 3 = insertion T beginning and insertion middle
    seq3 = seq0[:]
    seq3 = "T" + seq3
    seq3 = seq3[:4] + 'G' + seq3[4:]
    print(seq3)
    # seq 4 = double insertion AC end and insertion middle
    seq4 = seq0[:]
    seq4 = seq4 + "AC"
    seq4 = seq4[:4] + 'T' + seq4[4:]
    print(seq4)
    # global alignment
    align_star_multiple([seq0, seq1, seq2, seq3, seq4])

ModuleNotFoundError: No module named 'utils'